# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф.

Модель с максимально большим значением *accuracy*, нужно довести долю правильных ответов по крайней мере до 0.75. Проверить *accuracy* на тестовой выборке

## Открыть и изучить файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Загрузим данные из файла /datasets/users_behavior.csv. Скачать датасет
df = pd.read_csv('/datasets/users_behavior.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


Пропусков нет, уже хорошо. Названия столбцов нас устраивают.
Посмотрим на данные

In [2]:
print(df.head(5))

   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0


количество звонков и сообщений можно конечно перевести в целые значения и столбец is_ultra представляет критерий тарифа.

## Разбить данные на выборки

Разделим данные на "обучающую выборку", "валидационную выборку" и на "тестовую выборку" с разбивкой 60/20/20 % соответственно

In [9]:
df_train, df_valid_test = train_test_split(df, test_size=0.40, random_state=12345) # разбиваем 60/40
features_train = df_train.drop(['is_ultra'], axis=1) # обучающая выборка
target_train = df_train['is_ultra'] # целевая выборка

df_valid, df_test = train_test_split(df_valid_test, test_size=0.50, random_state=12345) # разбиваем оставшиеся 40% пополам
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

In [10]:
print('разммер обучающей выборки', df_train.shape)
print('разммер валидационной выборки', df_valid.shape)
print('разммер тестовой выборки', df_test.shape)

разммер обучающей выборки (1928, 5)
разммер валидационной выборки (643, 5)
разммер тестовой выборки (643, 5)


Исследуем модель решающего дерева

In [11]:
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
print(accuracy_score(target_valid, predictions))

0.713841368584759


Результат 0,71 недостаточен по нашему условию 0,75

Проверим на разном количествет глубины "дерева" и выберем лучшее

In [12]:
best_model = None
best_result = 0
best_depth = 0
for depth in range(1, 6):
    model = DecisionTreeClassifier(max_depth=depth, random_state=12345)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
print("Accuracy лучшей модели:", best_result, "глубина лучшей модели:", best_depth) 

Accuracy лучшей модели: 0.7853810264385692 глубина лучшей модели: 3


Это уже хорошо, с глубиной 3 стоит использовать эту модель

так же посмотрим на модель случайного леса

In [14]:
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(1, 15):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        result = model.score(features_valid, target_valid)
        if result > best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth

print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print('Количество деревьев', best_est)
print('Глубина дерева', best_depth)

Accuracy наилучшей модели на валидационной выборке: 0.8040435458786936
Количество деревьев 12
Глубина дерева 6


результат немного выше чем у предыдущей модели хоть и использования 12 деревьев увеличивает время работы модели

так же посмотрим на модель логистической регрессии

In [ ]:
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
result = model.score(features_valid, target_valid)

print("Accuracy модели логистической регрессии на валидационной выборке:", result)

показатель почти такойже как и у модели дерева что нас вполне устраивает

## Проверьте модель на тестовой выборке

Лучший результат показала модель "случайного леса". Проверим ее работу с выбранным параметром 12 на тестовой выборке.

In [16]:
model = RandomForestClassifier(random_state=12345, n_estimators=12)
model.fit(features_train, target_train)
result = model.score(features_test, target_test)
print("Accuracy модели логистической регрессии на тестовой выборке:", result)

Accuracy модели логистической регрессии на тестовой выборке: 0.7869362363919129


Отличный результат для нашей модели

## (бонус) Проверить модели на адекватность

In [15]:
from sklearn.dummy import DummyClassifier
model_dc = DummyClassifier(strategy='most_frequent', random_state=12345)
model_dc.fit(features_train, target_train)
result = model_dc.score(features_valid, target_valid)
result

0.7060653188180405

Значение Accuracy ниже значения выбранной нами модели что говорит об адекватности нашей модели.

<b>Вывод:</b> 

Для определения лучшей модели выбора тарифа разделили данные на 3 выборки: тренировочную, проверочную и тестовую. Проверили работу на 3-х моделях:
- Дерево решений,
- Слчайный лес,
- Линейная регрессия.
  
По результатам тестирования лучший результат показала модель "Случайный лес" с гиперпараметрами 12 деревьев и глубиной дерева 6.
    
    Проверка выбранной модели на тестовой выборке показала результат качества 0,7869 что вполне устраивает наше условие не меньше 0,75.
Эту модель и можно рекомендовать в качестве подбора тарифа.